# Reinforcement Learning - Markov Decision Process

> In this notebook, we will be implementing policy iteration and value iteration, which are both a dynamic programming algorthm used to find the optimal policy for an agent interacting with an environment which satisfies the Markov property. These lay the foundation of reinforcement learning. We will be testing our implementation on FrozenLake environment and some custom versions we made. Finally we will compare and try to draw conclusions.

## Importing libs

In [315]:
import gymnasium as gym
from gymnasium.envs.registration import register
from gymnasium.envs.toy_text.frozen_lake import FrozenLakeEnv
from gymnasium import spaces
import numpy as np
import time
import pprint

## Custom Environment



Here we are creating a new class for our custom environment with a 5*5 custom map and similar methods to the standard environments in the gymnasium package.

In [ ]:
class CustomFrozenLakeEnv(gym.Env):
    def __init__(self, slippery=False):
        self.size = 5
        self.n_states = self.size * self.size
        self.n_actions = 4
        self.slippery = slippery
        
        # Define custom 5x5 map
        self.desc = np.array([
            ['S', 'F', 'F', 'F', 'F'],
            ['F', 'H', 'F', 'H', 'F'],
            ['F', 'F', 'F', 'F', 'H'],
            ['H', 'F', 'F', 'F', 'F'],
            ['F', 'F', 'H', 'F', 'G']
        ], dtype='<U1')
        
        # Calculate positions
        self.start_pos = None
        self.goal_pos = []
        self.hole_pos = []
        for row in range(self.size):  ## idea was that there may be many goals but not doing that anymore
            for col in range(self.size):
                if self.desc[row, col] == 'S':
                    self.start_pos = (row, col)
                elif self.desc[row, col] == 'G':
                    self.goal_pos.append((row, col))
                elif self.desc[row, col] == 'H':
                    self.hole_pos.append((row, col))
        
        # first is row and second is coulumn
        self.actions = {
            0: (0, -1),    # Left
            1: (1, 0),     # Down
            2: (0, 1),     # Right
            3: (-1, 0)     # Up
        }
        
        # Build transition model
        self.P = self._make_transition_model()
        
        # Define spaces and state
        self.action_space = spaces.Discrete(self.n_actions)
        self.observation_space = spaces.Discrete(self.n_states)
        self.state = None

    def reset(self, seed=None, **kwargs):
        super().reset(seed=seed, **kwargs)
        self.state = self.pos_to_state(self.start_pos) #what;s the satte of the start position
        return self.state, {} # we ain't sending any info

    def step(self, action):
        current_pos = self.state_to_pos(self.state) # we are only storing state not position so this roundabout
        row, col = current_pos
        
        # if it is slippery then some stochastic flavour
        if self.slippery:
            action = self.np_random.choice([action, (action + 1) % 4, (action - 1) % 4])
        

        # calculus :)
        dr, dc = self.actions[action]
        new_row, new_col = row + dr, col + dc
        
        # Ensure within bounds
        new_row = np.clip(new_row, 0, self.size - 1)
        new_col = np.clip(new_col, 0, self.size - 1)
        new_pos = (new_row, new_col)
        new_state = self.pos_to_state(new_pos)
        
        # Check for hole or goal
        terminated = False
        reward = 0.0
        if new_pos in self.hole_pos:
            terminated = True
        elif new_pos in self.goal_pos:
            terminated = True
            reward = 1.0
        
        self.state = new_state #change state
        return new_state, reward, terminated, False, {}

    

    def _make_transition_model(self):
        P = {s: {a: [] for a in range(self.n_actions)} for s in range(self.n_states)} #initializing P[s][a] to an empty list
        for s in range(self.n_states):
            row, col = self.state_to_pos(s)
            # Check if current state is terminal
            current_pos = (row, col)
            if current_pos in self.hole_pos or current_pos in self.goal_pos:
                for a in range(self.n_actions):
                    # Terminal state: stay indefinitely with 0 reward
                    P[s][a] = [(1.0, s, 0.0, True)]
                continue  # Skip normal transition logic

            # Existing transition logic for non-terminal states
            for a in range(self.n_actions):
                outcomes = []
                actions_to_consider = ([a] if not self.slippery else [a, (a+1)%4, (a-1)%4])
                prob = 1.0 if not self.slippery else 1/3 # stochastic case
                for a2 in actions_to_consider:
                    #compute new state
                    dr, dc = self.actions[a2]
                    nr = np.clip(row + dr, 0, self.size - 1)
                    nc = np.clip(col + dc, 0, self.size - 1)
                    new_pos = (nr, nc)
                    ns = self.pos_to_state(new_pos)
                    #check and assign prob accordingly
                    done = new_pos in self.hole_pos or new_pos in self.goal_pos
                    reward = 1.0 if new_pos in self.goal_pos else 0.0
                    outcomes.append((prob, ns, reward, done))
                P[s][a] = outcomes
        return P

    # easy peasy
    def pos_to_state(self, pos):
        row, col = pos
        return row * self.size + col

    def state_to_pos(self, state):
        row = state // self.size
        col = state % self.size
        return (row, col)

This is another custom environment similar to FrozenLake with a 4*4 size except with a twist. The agent will be rewarded only when it collects key along its path to reach the goal. This forces the agent to adopt a particular route. Kind of like travelling in a traffic where you are required to visit a stop (say a petrol pump)

In [ ]:
class ExpandedFrozenLakeEnv(gym.Env):
    
    def __init__(self, slippery=False):
        self.size = 4
        self.n_states_base = self.size * self.size
        self.n_states = self.n_states_base * 2  # Double for key status
        self.n_actions = 4
        self.slippery = slippery
        
        # Define 4x4 map with key
        self.desc = np.array([
            ['S', 'F', 'F', 'F'],
            ['F', 'H', 'F', 'K'],
            ['F', 'F', 'F', 'F'],
            ['H', 'F', 'F', 'G']
        ], dtype='<U1')
        
        # Identify key, start, goal, and hole positions
        self.start_pos = None
        self.goal_pos = []
        self.hole_pos = []
        self.key_pos = None
        for row in range(self.size): # same idea as for the above class
            for col in range(self.size):
                if self.desc[row, col] == 'S':
                    self.start_pos = (row, col)
                elif self.desc[row, col] == 'G':
                    self.goal_pos.append((row, col))
                elif self.desc[row, col] == 'H':
                    self.hole_pos.append((row, col))
                elif self.desc[row, col] == 'K':
                    self.key_pos = (row, col)
        
        self.actions = {
            0: (0, -1),   # Left
            1: (1, 0),    # Down
            2: (0, 1),    # Right
            3: (-1, 0)    # Up
        }

        self.P = self._make_transition_model()

        self.action_space = spaces.Discrete(self.n_actions)
        self.observation_space = spaces.Discrete(self.n_states)
        # Initialize state
        self.state = None
        self.has_key = None

    def reset(self, seed=None, **kwargs):
        super().reset(seed=seed, **kwargs)
        self.has_key = False
        self.state = self.pos_to_state(self.start_pos)
        return self.get_full_state(), {}

    def step(self, action):
        current_pos = self.state_to_pos(self.state)
        row, col = current_pos
        
        # act 
        if self.slippery:
            action = self.np_random.choice([action, (action + 1) % 4, (action - 1) % 4])
        
        dr, dc = self.actions[action]
        new_row, new_col = row + dr, col + dc
        
        # Ensure within bounds
        new_row = np.clip(new_row, 0, self.size - 1)
        new_col = np.clip(new_col, 0, self.size - 1)
        new_pos = (new_row, new_col)
        new_state = self.pos_to_state(new_pos)
        
        # Check if key is collected
        if new_pos == self.key_pos:
            self.has_key = True
        
        # Check for hole or goal
        terminated = False
        reward = 0.0
        if new_pos in self.hole_pos:
            terminated = True
        elif new_pos in self.goal_pos and self.has_key:
            terminated = True
            reward = 1.0
        
        self.state = new_state
        full_state = self.get_full_state()
        return full_state, reward, terminated, False, {}

    def _make_transition_model(self):
        n_base = self.n_states_base  # 16 for 4x4 grid
        P = {s: {a: [] for a in range(self.n_actions)} for s in range(self.n_states)} #initialize P[s][a] to an empty list
        
        for s in range(self.n_states):
            base_state = s % n_base
            has_key = (s >= n_base)
            row, col = self.state_to_pos(base_state)
            current_pos = (row, col)

            # Terminal states: hole OR goal with key
            if current_pos in self.hole_pos or (current_pos in self.goal_pos and has_key):
                for a in range(self.n_actions):
                    P[s][a] = [(1.0, s, 0.0, True)]
                continue #skipping normal logic for now

            for a in range(self.n_actions):
                outcomes = []
                actions_to_consider = [a] if not self.slippery else [a, (a+1)%4, (a-1)%4]
                prob = 1.0 if not self.slippery else 1.0/len(actions_to_consider)
                
                for a2 in actions_to_consider:
                    #compute new state
                    dr, dc = self.actions[a2]
                    nr = np.clip(row + dr, 0, self.size-1)
                    nc = np.clip(col + dc, 0, self.size-1)
                    new_pos = (nr, nc)
                    new_base_state = self.pos_to_state(new_pos)
                    
                    # Update key status
                    new_has_key = has_key
                    if not has_key and new_pos == self.key_pos:
                        new_has_key = True
                    
                    next_state = new_base_state + (n_base * int(new_has_key))
                    
                    # Check terminal conditions
                    terminated = False
                    reward = 0.0
                    if new_pos in self.hole_pos:
                        terminated = True
                    elif new_pos in self.goal_pos and new_has_key:
                        terminated = True
                        reward = 1.0
                    
                    outcomes.append((prob, next_state, reward, terminated))
                
                P[s][a] = outcomes
        
        return P

    #trivial stuff
    def pos_to_state(self, pos):
        row, col = pos
        return row * self.size + col

    def state_to_pos(self, state):
        row = state // self.size
        col = state % self.size
        return (row, col)

    def get_full_state(self):
        return self.state + (self.n_states_base * int(self.has_key))

### Registering these custom environmnts

In [318]:
# Register environments
gym.register(
    id="CustomFrozenLake-v1",
    entry_point=CustomFrozenLakeEnv,
    kwargs={'slippery': False},
)

gym.register(
    id="ExpandedFrozenLake-v1",
    entry_point=ExpandedFrozenLakeEnv,
    kwargs={'slippery': False},
)

gym.register(
    id="CustomFrozenLake-v1-slip",
    entry_point=CustomFrozenLakeEnv,
    kwargs={'slippery': True},
)

gym.register(
    id="ExpandedFrozenLake-v1-slip",
    entry_point=ExpandedFrozenLakeEnv,
    kwargs={'slippery': True},
)

We will iterate over these environment ids to test.

In [319]:
# env ids to test
env_ids = ['FrozenLake-v1','CustomFrozenLake-v1','ExpandedFrozenLake-v1','CustomFrozenLake-v1-slip','ExpandedFrozenLake-v1-slip']  # frozenlake-v1 is for the default 

## Dynamic Programming Implementation

### Policy iteration

We will start off wth a random policy and evaluate the value function for all states according to this policy. This is known as Policy evaluation.

Next we will improve our policy through ${Q}$ functions. We will check if performing action ${a}$ and then following policy ${\pi}$  gives us a better value for that particular state than following ${\pi}$ from beginning.
 If that is so, then by policy improvement theorem, it is always better to choose this action ${a}$ instead of following ${\pi}$ when we are at that state.
 
This way we frame our new policy for all the states and then again evaluate the value function. This process is repeated again and again, and it finally converges when we reach the optimal value functon and optimal policy.

We can ensure that we have reached the optimal policy, provided there is no change in the value of all the states. Realistically though, we check if values have reached a certain tolerance.

#### Policy Evaluation

Below is the update equation for the values of our states which we need to implement in code. 

We will be going with the Gauss-Siedel updates which means that our list of values of each state will be updated in-place. That is instead of maintaining two lists, one for old and other for new, we will mantain a single list and use whatever values available to make the updates. This converges usually faster and consumes lesser space. 

Since we need to update each state, we will first run an outer loop on states. Then we see that the outer summation is over actions, so the next inner loop is over the set of all actions. The final summation is over the nest state and reward which we unpack from  env[s][a] and conclude our final loop. 


$$
\Large{\begin{aligned}
v_{k+1}(s) &\doteq \mathbb{E}_\pi \left[ R_{t+1} + \gamma v_k(S_{t+1}) \mid S_t = s \right] \\
          &= \sum_a \pi(a \mid s) \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v_k(s') \right]
\end{aligned}}
$$

In [320]:


def policy_evaluation(env, policy, nStates, nActions, discount=0.99, tolerance=1e-6):

    '''
    ------------------------------------------------------------------------------
    This function would return the value function for all states given 
    a policy and a model

    nStates is number of states
    nActions is number of actions

    policy[s] = give the probability distribution over all actions

    env[s][a] = is a dictionary of the form (prob, next_state, reward, done)
    means if you perform action a in state s ten these are the probabilities
    of moving to these states with this reward. done marks the end of the episode.
    
    tolerance would be used to determine when the value functions have converged.
    -------------------------------------------------------------------------------
    '''

    values = np.zeros(nStates) # initialize all the values as zero
    # side note: we can initialize them as we wish except that the
    # all the terminal states must be initialized zero

    while True:
        delta = 0
        for state in range(nStates):
            new_value_s = 0 #temporary var to store the result of computation since we are using a single array
            for action, action_prob in enumerate(policy[state]): # enumerate will return an index which we store in action variable
                for prob, nextState, reward, done in env[state][action]:
                    new_value_s += action_prob * prob * (reward + discount*values[nextState]) #update equation
            delta = max(delta, abs(new_value_s - values[state])) # the max difference between the new and old values
            values[state] = new_value_s
        if (delta < tolerance):
            break

    return values

#### Policy Improvement

We want to change our policy so that when we encounter the state $s$ we always choose the action which maximizes the $\Large{q_{\pi}}$ function. 

Since we need to change the policy for each action the outermost loop is over the set of states. Then we need to find the maximum over all action for which we create a numpy array and then loop over te set of actions. Finally the last loop is over the next state and reward which we again unpack from the environment. In the end, we return this new policy. 

$$
\Large{\begin{aligned}
\pi'(s) &\doteq \arg\max_a q_\pi(s, a) \\
       &= \arg\max_a \mathbb{E} \left[ R_{t+1} + \gamma v_\pi(S_{t+1}) \mid S_t = s, A_t = a \right] \\
       &= \arg\max_a \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v_\pi(s') \right]
\end{aligned}}
$$

In [321]:
def policy_improvement(env, nStates, nActions, values, discount=0.99):
    '''
    --------------------------------------------------------------------------------------
    This function would return a new policy given the value function and the environment

    nStates is number of states
    nActions is number of actions

    env[s][a] = is a dictionary of the form (prob, next_state, reward, done)
    means if you perform action a in state s ten these are the probabilities
    of moving to these states with this reward. done marks the end of the episode.
    ---------------------------------------------------------------------------------------
    '''

    policy = np.zeros([nStates, nActions]) # policy[s] would give probability distribution over all actions

    for state in range(nStates):
        q_states = np.zeros(nActions) #we will store all the q function list for each action here
        for action in range(nActions):
            for prob, state_next, reward, done in env[state][action]:
                q_states[action] += prob * (reward + discount * values[state_next])
        best_action = np.argmax(q_states)
        policy[state, best_action] = 1.0

    return policy
    
    

#### Policy Iteration (Implementation)

We will finally be interleaving both the evaluation and the improvement of our policy in this final iteration and checking if we have reached the optimal condition by seeing if the new policy we are getting through policy improvement is the same as our old policy, meaning there is no further improvement. 

In [322]:
def policy_iteration(env, nStates, nActions, discount=0.99, max_iter = 1000):
    policy = np.ones([nStates, nActions]) / nActions # initialize uniform random policy

    for iter in range(max_iter):
        values = policy_evaluation(env, policy, nStates, nActions, discount)
        new_policy = policy_improvement(env, nStates, nActions, values, discount)

        if np.all(new_policy == policy):
            break
        policy = new_policy

    return policy, values, iter+1

### Value Iteration

Value iteration is another way of finding the optimal policy in the realm of dynamic programming. What we do here is choose the best possible value possible for that state by iterating over all possible actions. This way the value function converges to the optimal value function.  

Then we extract the optimal policy from this optimal value function by iterating over all actions for all states and seeing which one maximizes the function. We then assign $\Large{\pi(s) = a}$


$$
\Large{\begin{aligned}
v_{k+1}(s) &= \max_a \sum_{s', r} p(s', r \mid s, a) \left[ r + \gamma v_k(s') \right]
\end{aligned}}
$$



In [323]:
def value_iteration(env, nStates, nActions, discount=0.99, tolerance=1e-6):
    '''
    --------------------------------------------------------------------------------------
    This function would return the most optimal policy and the value function given an 
    environment

    nStates is number of states
    nActions is number of actions

    env[s][a] = is a dictionary of the form (prob, next_state, reward, done)
    means if you perform action a in state s ten these are the probabilities
    of moving to these states with this reward. done marks the end of the episode.
    ---------------------------------------------------------------------------------------
    '''

    values = np.zeros(nStates)

    while True:
        delta = 0
        for state in range(nStates):
            q_states = np.zeros(nActions) #store all the q function values for all possible action
            for action in range(nActions):
                for prob, nextState, reward, done in env[state][action]:
                    q_states[action] += prob * (reward + discount * values[nextState])
            max_q = max(q_states)
            delta = max(delta, abs(max_q - values[state]))
            values[state] = max_q  # choose the best one and save it
        if (delta < tolerance):
            break

    # extracting optimal policy from the optimal value function
    policy = np.zeros([nStates, nActions])
    for state in range(nStates):
        q_states = np.zeros(nActions)
        for action in range(nActions):
            for prob, nextState, reward, done in env[state][action]:
                q_states[action] += prob * (reward + discount * values[nextState]) 
        best_action = np.argmax(q_states) #choose the action maximizing the q function 
        policy[state, best_action] = 1.0
    return policy, values

## Policy Evaluation

Having made our optimal policy through policy and value iteration methods, we would like to test our policy in the environment to see how it performs. 

In [324]:
def evaluate_policy(env, policy, nEpisodes=1000, max_steps=100):
    '''
    --------------------------------------------------------------------------------------
    This function would return the mean reward and the mean number of steps taken over
    all the episodes

    nEpisodes is number of episodes
    max_steps is the max number of steps to be taken in the environment

    env[s][a] = is a dictionary of the form (prob, next_state, reward, done)
    means if you perform action a in state s ten these are the probabilities
    of moving to these states with this reward. done marks the end of the episode.

    policy[s] give the probability distribution over all actions
    ---------------------------------------------------------------------------------------
    '''
    rewards = []
    lengths = []
    for _ in range(nEpisodes):
        obs, _ = env.reset() #first observation
        done = False
        ep_reward = 0
        steps = 0
        while not done and steps < max_steps: #till either goal is reached or max_steps are taken
            action = np.argmax(policy[obs])
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            ep_reward += reward
            steps += 1

        rewards.append(ep_reward)
        lengths.append(steps)

    return np.mean(rewards), np.mean(lengths)
        


## Model

We will now test our algorithm in three environment of FrozenLake and see the results of our DP algorithm

In [325]:
results = {}
for env_id in env_ids:
    env = gym.make(env_id)
    env_model = env.unwrapped.P  # transition model
    nStates, nAction = env.observation_space.n, env.action_space.n

    # Value Iteration
    t0 = time.time()
    policy_VI, values_VI = value_iteration(env_model, nStates, nAction)
    t_vi = time.time() - t0
    reward_VI, length_VI = evaluate_policy(env, policy_VI)

    # Policy Iteration
    t0 = time.time()
    policy_PI, values_PI, iters = policy_iteration(env_model, nStates, nAction)
    t_pi = time.time() - t0
    reward_PI, length_PI = evaluate_policy(env, policy_PI)

    results[env_id] = {
        'VI': {'time': t_vi, 'reward': reward_VI, 'length': length_VI},
        'PI': {'time': t_pi, 'reward': reward_PI, 'length': length_PI, 'iters': iters}
    }

pprint.pprint(results)

{'CustomFrozenLake-v1': {'PI': {'iters': 3,
                                'length': np.float64(8.0),
                                'reward': np.float64(1.0),
                                'time': 0.0032219886779785156},
                         'VI': {'length': np.float64(8.0),
                                'reward': np.float64(1.0),
                                'time': 0.0006968975067138672}},
 'CustomFrozenLake-v1-slip': {'PI': {'iters': 3,
                                     'length': np.float64(53.676),
                                     'reward': np.float64(0.576),
                                     'time': 0.046004295349121094},
                              'VI': {'length': np.float64(54.257),
                                     'reward': np.float64(0.523),
                                     'time': 0.02655339241027832}},
 'ExpandedFrozenLake-v1': {'PI': {'iters': 3,
                                  'length': np.float64(6.0),
                                 

# Results Summary

Here's a detailed overview of the results comparing Policy Iteration (PI) and Value Iteration (VI) across different FrozenLake environments:

## Performance Comparison

| Environment | Algorithm | Iterations | Time (s) | Avg. Reward | Avg. Length |
|-------------|-----------|------------|----------|-------------|-------------|
| **CustomFrozenLake-v1**<br>(5×5, no slip) | PI | 3 | 0.0032 | 1.0 | 8.0 |
| | VI | - | 0.0007 | 1.0 | 8.0 |
| **CustomFrozenLake-v1-slip**<br>(5×5, slippery) | PI | 3 | 0.0460 | 0.576 | 53.7 |
| | VI | - | 0.0266 | 0.523 | 54.3 |
| **ExpandedFrozenLake-v1**<br>(4×4 + key, no slip) | PI | 3 | 0.0061 | 1.0 | 6.0 |
| | VI | - | 0.0007 | 1.0 | 6.0 |
| **ExpandedFrozenLake-v1-slip**<br>(4×4 + key, slippery) | PI | 4 | 0.0458 | 1.0 | 25.2 |
| | VI | - | 0.0140 | 0.998 | 25.6 |
| **FrozenLake-v1**<br>(Default 4×4) | PI | 3 | 0.0257 | 0.713 | 45.5 |
| | VI | - | 0.0192 | 0.741 | 44.8 |

## Key Insights

1. **Perfect Performance in Non-Slippery Environments**:
   - Both algorithms achieved 100% success rate (reward=1.0) in all non-slippery environments
   - VI consistently executed faster (3-10× speedup)

2. **Slippery Environment Performance**:
   - Expanded environment maintained near-perfect performance (99.8-100%)
   - Custom environment showed moderate success (52-58%)
   - Original FrozenLake had 71-74% success rate

3. **Path Efficiency**:
   - Non-slippery environments had optimal paths (6-8 steps)
   - Expanded environment maintained relatively efficient paths (~25 steps) even when slippery
   - Original FrozenLake had longest paths (~45 steps)

4. **Algorithm Comparison**:
   - VI consistently executed faster than PI (30-70% reduction)
   - Both algorithms converged quickly (3-4 iterations)
   - Performance differences were minimal in all the environments

5. **Environment Complexity**:
   - Key mechanics in expanded environment were successfully navigated
   - Larger 5×5 grid showed more performance degradation when slippery
   - Original FrozenLake proved most challenging despite smaller size (on the basis of average length)

## Conclusion

The DP algorithms demonstrated awesome performance in deterministic environments, with VI being computationally faster. In stochastic environments, performance varied based on map design, with the expanded key-based environment showing very less drop in reward to slippage. The original FrozenLake environment required the most length on average and thus proved more challenging for the agent despite being smaller.